In [ ]:
if (!require("tidyverse")) install.packages("tidyverse")
library(tidyverse)
#config
#output dirs, relative to this notebook
inputFolder <- "dataForRScrtips"
output_dir<- "OutputImages"
inputFilename <- "JoshuaTree_decadal_anomalies_combined.csv"
input_filepath <-file.path(inputFolder, inputFilename)
#baseline period referefce (used in Python script for plot subtitle)
baselineText <- "relative to 1995 -2014 baseeline"
#iff output dir doesn't exist, create it
if (!dir.exists(output_dir)) {
  dir.create(output_dir)
  print(paste("made directory:", output_dir))
} else {
  print(paste(" directory already E @:", output_dir))
}
#define CMIP6 standard colors ->seems to be how the paper did it
    #define colours only for future scarios SSPs
    #historical data will also be plotted manually in black or grey
    #got colours from Hex colour picker from the paper, maybe M + L have thoughts on how standard this is?
ssp_colors <- c("SSP 5-8.5" ="#840312", #dark red for high emissisions
  "SSP 3-7.0"= "#D55E00", #orangey red
  "SSP 2-4.5" ="#377eb8", #Blue
  "SSP 1-2.6" = "#05314d")  #Dark Blue for low emissiosn
#data loading and preip

#does input file exist, if it does load the data
if (!file.exists(input_filepath)) {
  stop(paste("input file file not found:", input_filepath, ", need some python data generating scfipt to run first"))}
print(paste("data read @:", input_filepath))
# read_csv & suppress column type messages
data <- read_csv(input_filepath, show_col_types =FALSE)
# Convert categorical columns to factors and ensure Year is numeric
#convert categorical cols to factors and esnure that year is numeric
data <-data%>%
  mutate(Scenario = as.factor(Scenario),Simulation = as.factor(Simulation),Variable = as.factor(Variable),Year = as.numeric(Year))
#calculate ensemble means (multi-model means
#get the mean across simulations for each year, scenario and then variabel
ensembleMeans <-data%>%
  group_by(Year, Scenario,Variable) %>%
  summarise(MeanAnomaly = mean(Anomaly, na.rm =TRUE), .groups = 'drop')
#plot functions
createTimeSeriesPLots <-function(dfIndividual, dfMreans, varName, yLab) {
  #filter data for specified var
  plotdataIndividual <-dfIndividual %>% filter(Variable == varName)
  plotDataMeans<- dfMreans %>% filter(Variable == varName)
  #from papers, historical data needs to be in diff colour than future scnarios?
  #so seperate historical data from teh rest
  historicalIndividual <-plotdataIndividual %>% filter(Scenario == "Historical Climate")
  historicalMeans <- plotDataMeans %>%filter(Scenario == "Historical Climate")
  # Separate Future data (SSPs)
  future_individual <-plotdataIndividual %>% filter(Scenario != "Historical Climate")
  future_means <-plotDataMeans %>% filter(Scenario !="Historical Climate")

  #start the plot, ORDER MATTERS for layering uhg!
  p <-ggplot() +
    # okay first, historical individual sims in thin grey
    geom_line(data =historicalIndividual, aes(x = Year,y = Anomaly, group =Simulation), 
              colcolouror = "grey", linewidth = 0.2,alpha =0.6) +
    #then future individual simulations are thin
    #interaction() for grouping is good as saftey iff simulations names overlap across scenarios
    geom_line(data = future_individual, aes(x =Year, y =Anomaly, colour = Scenario, group = interaction(Scenario, Simulation)), 
              linewidth = 0.2, alpha = 0.5) +
    #historical ensemble just in a fat black line
    geom_line(data = historicalMeans, 
              aes(x =Year, y = MeanAnomaly), 
              colour= "black", linewidth =1.2) +
    #future ensemble means shoudl be thick and coloured
    geom_line(data =future_means, 
              aes(x = Year, y = MeanAnomaly, colour =Scenario), 
              linewidth= 1.5)+
    #styping and labels
    geom_hline(yintercept = 0,linetype = "solid", color ="black",linewidth=0.5) +
    #apply colour scale to SSP scenarios (only!)
    scale_color_manual(values = ssp_colors, breaks = names(ssp_colors)) +
    labs(title = paste("Per Decade Mean Annual", varName, "Anomalies (JTree)"),
         subtitle = baselineText,
         x= "Time (yr)",
         y=yLab)+   
    #apparently pubs use theme_bw ? idk if that's true tbh
    theme_bw(base_size =14)+ 
    theme(legend.position = "bottom",
          plot.title = element_text(hjust = 0.5,face ="bold"),
          plot.subtitle =element_text(hjust =0.5),
          panel.grid.minor =element_blank())+
    #x axis limits nad breaks ithink will make thislookless messy?
    #or idk if lil gap is good or if should be 1980?
    #this is hardcoded but maybe for more fleixbilty could set this to min and max year data?
    #TODO^^
    scale_x_continuous(breaks = seq(1979, 2100,by =20),limits =c(1979, 2100))
  return(p)
}

#gen and save plots now that funcs are defined !

#Preip plot
if ("precip" %in% levels(data$Variable)) {
  pPrecip <- createTimeSeriesPLots(data,ensembleMeans, "precip","Precipitation Change (%)")
  #save plot
  outputPreipFileName <-file.path(output_dir,"JTreePrepTimeSeries.png")
  ggsave(outputPreipFileName, plot = pPrecip, width = 10, height =6, dpi = 300)
  print(paste("saved plot at:", outputPreipFileName))
} else {
  print("prei data not in Ccsv, check it")
}
#temp plot
if ("temp" %in% levels(data$Variable)) {
  # Assuming Python converted K to C, the anomaly unit is Δ°C
  pTemp <-createTimeSeriesPLots(data, ensembleMeans, "temp", "Temperature Change (Δ°C)")
  output_filename_temp <- file.path(output_dir, "jTreeTempTimeseriesPLot.png")
  ggsave(output_filename_temp, plot = pTemp, width = 10, height =6, dpi = 300)
  print(paste("Saved plot to:", output_filename_temp))
} else {
  print("temp data wasn't found w/in csv")
}


[1] " directory already E @: OutputImages"
[1] "data read @: dataForRScrtips/JoshuaTree_decadal_anomalies_combined.csv"


Warning message in geom_line(data = historicalIndividual, aes(x = Year, y = Anomaly, :
“Ignoring unknown parameters: `colcolouror`”


[1] "saved plot at: OutputImages/JTreePrepTimeSeries.png"


Warning message in geom_line(data = historicalIndividual, aes(x = Year, y = Anomaly, :
“Ignoring unknown parameters: `colcolouror`”


[1] "Saved plot to: OutputImages/jTreeTempTimeseriesPLot.png"
